In [1]:
# Lan Luo, CPSC 437, Database Project
# This file constructs a database from downloaded raw HTML files from the Yale Facebook for each college
# that compiles basic information on each Yale students (first name, last name, class year, college, address,
# country, major, and birthday) into a single relation.

In [2]:
# import HTML parser/processor
from bs4 import BeautifulSoup
# import to use for regex
import re

In [27]:
collegeraw = None
college = None
collegeraw = open('Silliman.html').read()
college = BeautifulSoup(collegeraw, 'html.parser')

In [54]:
classes = [student for student in college.find_all("div", class_="student_container")]

In [55]:
students = []
for x in classes:
    students.append(str(x))
for i in range(len(students)):
    # remove extra html tags that won't be used to split the data
    students[i] = students[i].replace("</div>","")
    # replace html ampersand to &
    students[i] = students[i].replace("&amp;","&")
    # remove phone numbers (extraneous info - location information can be drawn from the home address)
    students[i] = re.sub('<br/>(\d{3} ?-)?\d{3}-\d{4} / ', "", students[i])
    # remove strange sequences of numbers (e.g. ending in YS), not address information so extraneous
    students[i] = re.sub('<br/>#? ?\d* YS', "", students[i])
    students[i] = re.sub('<br/>\d-\d{4} /', "", students[i])
    students[i] = re.sub('<br/>\d+ /', "", students[i])
    # if there's an american zip code, then add United States as a country to be consistent with students who have information for their foreign home country
    USzip = re.search('([A-Z]{2} \d{5}-?(\d{4})?)', students[i])
    if USzip:
        students[i] = students[i].replace(USzip.group(1), USzip.group(1) + "<br/>United States")    

In [95]:
out = []
for i in range(len(students)):
    out.append(students[i].split("<br/>"))
    
set(len(i) for i in out)

{3, 4, 5, 6, 7}

In [96]:
out[0]

['<div class="student_container"><div class="student_img"><img alt="" height="120" src="/facebook/Photo?id=412684" width="100"/><div class="year_container"><div class="student_year student_year_0">\'22<div class="year_border">  <div class="student_text_container"><div class="student_name"><h5 class="yalehead">Ababio, Rachel </h5><div class="student_info"><a class="email" href="mailto:rachel.ababio@yale.edu">rachel.ababio@yale.edu</a>',
 'SM-E52A ',
 '385 Camp St',
 'Platteville, WI 53818-1706',
 'United States',
 'Undeclared',
 'Jul 29']

In [97]:
fin = [i for i in out if len(i) == 7]
copy_fin = fin[:] 

for i in range(len(copy_fin)):
    yearsearch, emailsearch, lastsearch, firstsearch = None, None, None, None
    
    yearsearch = re.search('(\d{2})<div class="year_border">', copy_fin[i][0])
    if yearsearch:
        Year = "20" + yearsearch.group(1)
    emailsearch = re.search('href="mailto:(.*)">', copy_fin[i][0])
    if emailsearch:
        Email = emailsearch.group(1)
           
    lastsearch = re.search('<h5 class="yalehead">(.*),', copy_fin[i][0])
    if lastsearch:
        LastName = lastsearch.group(1)
    
    firstsearch = re.search('<h5 class="yalehead">.*, (.*) </h5>', copy_fin[i][0])
    if firstsearch:
        FirstName = firstsearch.group(1)
        
    del copy_fin[i][0]
    copy_fin[i].insert(0, "Benjamin Franklin")
    copy_fin[i].insert(0, Year)
    copy_fin[i].insert(0, Email)
    copy_fin[i].insert(0, LastName)
    copy_fin[i].insert(0, FirstName)
    # remove college tag at beginning of room sequence
    tagindex = (copy_fin[i][5]).find('-')
    copy_fin[i][5] = copy_fin[i][5][tagindex + 1:]

In [100]:
matchroomfin = []
for i in range(len(copy_fin)):
    if len(copy_fin[i][5]) <= 6:
        if re.search('[A-Z][0-9]{2}[A-Z]?', copy_fin[i][5]):
            matchroomfin.append(copy_fin[i])
        if re.search('\d{3}[A-Z]?', copy_fin[i][5]):
            matchroomfin.append(copy_fin[i])

In [102]:
len(matchroomfin)

320